In [36]:
import keras.callbacks
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers import Dropout
%matplotlib inline
import tensorflow as tf
import warnings

warnings.filterwarnings('ignore')
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras import layers, models

In [37]:
def prepareData():
    allData = pd.read_csv("data/songs_with_genre.csv", index_col=0,
                          header=[0, 1, 2])
    generes = allData["genre_id"]
    genres_index = pd.read_csv("data/genres_index.csv", index_col=0)
    allData.drop(columns=["genre_id"], inplace=True)
    data_x = allData
    data_x.columns = range(data_x.shape[1])
    #normalizing
    mu = np.mean(data_x, axis=0)
    sigma = (np.std(data_x, axis=0))
    data_x = (data_x - mu) / sigma
    #Creating the right lables
    generes.columns = range(generes.shape[1])
    targets = OneHotEncoder().fit_transform(genres_index).toarray()
    generes[0] = generes[0] - 1
    data_y = [targets[i - 1] for i in generes[0]]
    generes[0] = data_y

    test = generes.copy()
    test.rename(columns={0: "oneHotVector"}, inplace=True)
    test.shape
    split = pd.DataFrame(test["oneHotVector"].to_list(), columns=range(161))
    split.index = generes.index
    generes = split
    return data_x, generes

In [38]:
data_x, labels = prepareData()
x_train, x_rest, y_train, y_rest = train_test_split(data_x, labels, train_size=0.8)
x_test, x_valid, y_test, y_valid = train_test_split(x_rest, y_rest, train_size=0.5)
print(x_train.shape), print(len(y_train))
print(x_valid.shape), print(len(y_valid))
print(x_test.shape), print(len(y_test))

(39678, 518)
39678
(4960, 518)
4960
(4960, 518)
4960


(None, None)

In [39]:
n_features, n_outputs = x_train.shape[1], y_train.shape[1]
CNN = models.Sequential()
CNN.add(layers.Conv1D(filters=128, kernel_size=3, activation='relu',
                      input_shape=(n_features, 1), padding='same'))
CNN.add(layers.MaxPooling1D(pool_size=2))
CNN.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
CNN.add(layers.MaxPooling1D(pool_size=2))
CNN.add(layers.Conv1D(filters=128, kernel_size=3, activation='relu'))
CNN.add(layers.Dropout(0.6))  #0.6
CNN.add(layers.MaxPooling1D(pool_size=2))
CNN.add(layers.Flatten())
CNN.add(layers.Dense(128, activation='relu'))
CNN.add(layers.Dense(n_outputs, activation='softmax'))
CNN.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model\assets


INFO:tensorflow:Assets written to: saved_model/my_model\assets


In [40]:
# n_features, n_outputs = x_train.shape[1], y_train.shape[1]
# CNN = models.Sequential()
# CNN.add(layers.Conv1D(filters=32, kernel_size=5, activation='relu',
#                       input_shape=(n_features, 1), padding='same'))
# CNN.add(layers.MaxPooling1D(pool_size=4))
# CNN.add(layers.Conv1D(filters=128, kernel_size=3, activation='relu'))
# CNN.add(layers.MaxPooling1D(pool_size=4))
# CNN.add(layers.Dropout(0.6))
# CNN.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
# CNN.add(layers.MaxPooling1D(pool_size=4))
# CNN.add(layers.Dropout(0.3))
# CNN.add(layers.Flatten())
# CNN.add(layers.Dense(64, activation='relu'))
# CNN.add(layers.Dense(n_outputs, activation='softmax'))
# CNN.save('saved_model/my_model')

In [41]:
CNN.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_18 (Conv1D)          (None, 518, 128)          512       
                                                                 
 max_pooling1d_18 (MaxPoolin  (None, 259, 128)         0         
 g1D)                                                            
                                                                 
 conv1d_19 (Conv1D)          (None, 257, 64)           24640     
                                                                 
 max_pooling1d_19 (MaxPoolin  (None, 128, 64)          0         
 g1D)                                                            
                                                                 
 conv1d_20 (Conv1D)          (None, 126, 128)          24704     
                                                                 
 dropout_11 (Dropout)        (None, 126, 128)         

In [42]:
CNN.compile(optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy'])

In [43]:
callback = [keras.callbacks.EarlyStopping(monitor='accuracy', mode='max', patience=15)]
training = CNN.fit(x_train, y_train, epochs=30, validation_data=(x_valid, y_valid), batch_size=389, callbacks=callback)

Epoch 1/30
102/102 [==============================] - 53s 508ms/step - loss: 1.6452 - accuracy: 0.5115 - val_loss: 1.2948 - val_accuracy: 0.6194

In [44]:
# _, accuracy = CNN.evaluate(x_valid, y_valid, verbose=0)
# print(accuracy)
metrics_df = pd.DataFrame(training.history)
metrics_df[["loss","val_loss"]].plot()
metrics_df[["accuracy", "val_accuracy"]].plot()

In [45]:
_, accuracy_test = CNN.evaluate(x_test, y_test, verbose=0)
print(accuracy_test)

0.6830645203590393


In [46]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report

ypred = CNN.predict(x_test)
y_pred = (ypred > 0.5)
y = y_test.values
y = (y > 0.5)
# print(y)
# print(y_pred)

155/155 [==============================] - 2s 13ms/step

In [47]:
print('Accuracy score : ', accuracy_score(y, y_pred))
print('Precision score : ', precision_score(y, y_pred, average='weighted'))
print('Recall score : ', recall_score(y, y_pred, average='weighted'))
print('F1 score : ', f1_score(y, y_pred, average='weighted'))
print(classification_report(y_test, y_pred))

Accuracy score :  0.6171370967741936
Precision score :  0.7402672853869393
Recall score :  0.6171370967741936
F1 score :  0.6617008185270319
              precision    recall  f1-score   support

           0       1.00      0.08      0.15        12
           1       0.57      0.25      0.34        69
           2       0.83      0.82      0.82       123
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.96      0.96      0.96        53
           6       0.00      0.00      0.00        16
           7       0.40      0.09      0.14       219
           8       0.00      0.00      0.00         0
           9       0.84      0.77      0.81      1415
          10       0.00      0.00      0.00         3
          11       0.75      0.17      0.27        18
          12       0.75      0.61      0.67       944
          13       0.00      0.00      0.00         0
          14       0.65      0.48      0.55     